<a href="https://colab.research.google.com/github/bcury/Machine-Learning-Projects/blob/main/ML_spark_Associa%C3%A7%C3%A3o_FP_Growth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Modelo simples

In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 43 kB/s 
     |████████████████████████████████| 198 kB 81.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=6e6620cef193009f70321e5368575e4362d5a37c2c8dfd8a6e856fdd1c13d709
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pyspark, findspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName('associacao').getOrCreate()

In [4]:
from pyspark.ml.fpm import FPGrowth

In [5]:
from pyspark.sql.functions import split


In [10]:
transações = spark.read.text('Transacoes.txt').select(split('value','\s+').alias('transações'))
transações.show(truncate=False)

+-------------------------+
|transações               |
+-------------------------+
|[Cerveja, Pizza, Sorvete]|
|[Pizza, Sorvete]         |
|[Cerveja, Pizza]         |
|[Cerveja, Pizza, Sorvete]|
|[Cerveja, Pizza]         |
|[Pizza]                  |
+-------------------------+



In [11]:
fp = FPGrowth(itemsCol='transações',minSupport=0.5, minConfidence=0.5)

In [12]:
modelo = fp.fit(transações)
modelo.freqItemsets.show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+----------------+----+
|           items|freq|
+----------------+----+
|       [Sorvete]|   3|
|[Sorvete, Pizza]|   3|
|       [Cerveja]|   4|
|[Cerveja, Pizza]|   4|
|         [Pizza]|   6|
+----------------+----+



In [13]:
modelo.associationRules.show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+----------+----------+------------------+----+------------------+
|antecedent|consequent|        confidence|lift|           support|
+----------+----------+------------------+----+------------------+
| [Cerveja]|   [Pizza]|               1.0| 1.0|0.6666666666666666|
| [Sorvete]|   [Pizza]|               1.0| 1.0|               0.5|
|   [Pizza]| [Sorvete]|               0.5| 1.0|               0.5|
|   [Pizza]| [Cerveja]|0.6666666666666666| 1.0|0.6666666666666666|
+----------+----------+------------------+----+------------------+



quem compra cerveja, comra pizza, confinaa de 100%, etc

In [14]:
modelo.transform(transações).show(truncate=False)

+-------------------------+------------------+
|transações               |prediction        |
+-------------------------+------------------+
|[Cerveja, Pizza, Sorvete]|[]                |
|[Pizza, Sorvete]         |[Cerveja]         |
|[Cerveja, Pizza]         |[Sorvete]         |
|[Cerveja, Pizza, Sorvete]|[]                |
|[Cerveja, Pizza]         |[Sorvete]         |
|[Pizza]                  |[Sorvete, Cerveja]|
+-------------------------+------------------+



previsão acima: quem compra pizzaa e sorvete, tbm deve comprar cerveja, quem compra pizza, deve comprar sorvte e cerveja, etc....